# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#read csv data of citydata
citidata=pd.read_csv("../output_data/cities2.csv")
citydata=citidata.drop(columns=["Unnamed: 0"])
citydata.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mount Gambier,-37.83,140.77,53.60,87,90,8.05,AU,1619183172
1,Dwārka,22.24,68.97,83.71,70,100,15.90,IN,1619183304
2,Talnakh,69.49,88.40,33.39,89,89,14.34,RU,1619183305
3,Mataura,-46.19,168.86,48.00,95,34,1.99,NZ,1619183306
4,Kangaatsiaq,68.31,-53.46,26.15,75,85,7.76,GL,1619183306


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#store lat lng into locations
locations=citydata[["Lat","Lng"]].astype(float)

#convert humidity data into float
humidity=citydata["Humidity"].astype(float)


In [6]:
#create heatmap with humidity layer
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,
                               dissipating=False,max_intensity=100,
                               point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#set ideal weather condition: 70<max temp<80 and wind speed<10 and cloudiness=0,drop other unsatisfied condition rows
newcity=citydata.loc[(citydata["Max Temp"]<80)&(citydata["Max Temp"]>70)&(citydata["Wind Speed"]<10)&(citydata["Cloudiness"]==0)]
newcity

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
153,Port Hedland,-20.32,118.57,75.20,53,0,6.91,AU,1619183370
154,Saravan,39.72,45.65,77.00,19,0,4.61,AM,1619183371
317,Kuito,-12.38,16.93,73.40,56,0,6.91,AO,1619183475
372,Saldanha,-33.01,17.94,73.40,72,0,9.22,ZA,1619183499
434,São João da Barra,-21.64,-41.05,78.80,57,0,8.05,BR,1619183519
466,Tursunzoda,38.51,68.23,73.40,21,0,5.70,TJ,1619183530
476,Diamantina,-18.25,-43.60,72.93,49,0,2.01,BR,1619183533
489,Tura,25.52,90.22,77.88,31,0,4.32,IN,1619183537
523,Champerico,14.30,-91.92,75.20,88,0,4.32,GT,1619183548


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create new dataframe of hotel_df
hoteldf=newcity.reset_index()
#add column of "Hotel Name"
hoteldf.loc[:,"Hotel Name"]=""
#drop "index" column
hotel_df=hoteldf.drop(columns=["index"])
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Port Hedland,-20.32,118.57,75.20,53,0,6.91,AU,1619183370,
1,Saravan,39.72,45.65,77.00,19,0,4.61,AM,1619183371,
2,Kuito,-12.38,16.93,73.40,56,0,6.91,AO,1619183475,
3,Saldanha,-33.01,17.94,73.40,72,0,9.22,ZA,1619183499,
4,São João da Barra,-21.64,-41.05,78.80,57,0,8.05,BR,1619183519,
5,Tursunzoda,38.51,68.23,73.40,21,0,5.70,TJ,1619183530,
6,Diamantina,-18.25,-43.60,72.93,49,0,2.01,BR,1619183533,
7,Tura,25.52,90.22,77.88,31,0,4.32,IN,1619183537,
8,Champerico,14.30,-91.92,75.20,88,0,4.32,GT,1619183548,


In [13]:
#find the closest hotel for each city
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#use itterrows to iterate through dataframe
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
   #set parameters
    params={
        "location":f"{lat},{lng}",
        "types": "lodging",
        "radius":"5000",
        
        "key":g_key}
    
    
    #combine base_url and parameters, make api requests
    response=requests.get(base_url,params=params).json()
    #extract results
    result=response['results']
    
    try:
        
        print(f"The closest hotel is {result[0]['name']}.")
        hotel_df.loc[index,"Hotel Name"]=result[0]['name']
    except(KeyError, IndexError):
        print("Missing Field")
#display dataframe
hotel_df

The closest hotel is The Esplanade Hotel.
Missing Field
The closest hotel is Herdeiros de Antunes Capusso.
The closest hotel is Blue Bay Lodge.
The closest hotel is Pousada Porto De Canoas.
The closest hotel is Stoyanka Charli.
The closest hotel is POUSADA OURO DE MINAS.
The closest hotel is Hotel RIKMAN Continental.
The closest hotel is Hotel y restaurante "El Diamante".


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Port Hedland,-20.32,118.57,75.20,53,0,6.91,AU,1619183370,The Esplanade Hotel
1,Saravan,39.72,45.65,77.00,19,0,4.61,AM,1619183371,
2,Kuito,-12.38,16.93,73.40,56,0,6.91,AO,1619183475,Herdeiros de Antunes Capusso
3,Saldanha,-33.01,17.94,73.40,72,0,9.22,ZA,1619183499,Blue Bay Lodge
4,São João da Barra,-21.64,-41.05,78.80,57,0,8.05,BR,1619183519,Pousada Porto De Canoas
5,Tursunzoda,38.51,68.23,73.40,21,0,5.70,TJ,1619183530,Stoyanka Charli
6,Diamantina,-18.25,-43.60,72.93,49,0,2.01,BR,1619183533,POUSADA OURO DE MINAS
7,Tura,25.52,90.22,77.88,31,0,4.32,IN,1619183537,Hotel RIKMAN Continental
8,Champerico,14.30,-91.92,75.20,88,0,4.32,GT,1619183548,"Hotel y restaurante ""El Diamante"""


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Lat", "Lng"]]


In [14]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Lat", "Lng"]]
# Add marker layer ontop of heat map


#create hotel symbol layer
hotel_layer=gmaps.symbol_layer(
              locations,scale=4,
              info_box_content=hotel_info)
# Display figure
fig=gmaps.figure()
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
#create a combined map
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))